## Gain and miss. noise params

author: steeve.laquitaine@epfl.ch

In [2]:
# import libs
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

import os
import multiprocessing
import torch
import numpy as np
from matplotlib import pyplot as plt
import spikeinterface as si
import pandas as pd
import seaborn as sns

# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023/"
os.chdir(PROJ_PATH)

from src.nodes import utils
from src.nodes.utils import get_config
from src.nodes.validation import snr
from src.nodes.validation import amplitude as amp

SF_V = 20000
SF_S = 20000

# denser probe (horvath)  ------
# vivo (probe 1)
cfg_hv1, _ = get_config("vivo_horvath", "probe_1").values()
RAW_PATH_hv1 =  cfg_hv1["raw"]
PREP_PATH_hv1 = cfg_hv1["preprocessing"]["output"]["trace_file_path"]
# probe 2
cfg_hv2, _ = get_config("vivo_horvath", "probe_2").values()
RAW_PATH_hv2 =  cfg_hv2["raw"]
PREP_PATH_hv2 = cfg_hv2["preprocessing"]["output"]["trace_file_path"]
# probe 3
cfg_hv3, _ = get_config("vivo_horvath", "probe_3").values()
RAW_PATH_hv3 =  cfg_hv3["raw"]
PREP_PATH_hv3 = cfg_hv3["preprocessing"]["output"]["trace_file_path"]

# biophy. model
# (probe 1)
cfg_hs1, _ = get_config("silico_horvath", "concatenated/probe_1").values()
RAW_PATH_hs1 = cfg_hs1["dataeng"]["campaign"]["output"]["trace_file_path"]
PREP_PATH_hs1 = cfg_hs1["preprocessing"]["output"]["trace_file_path"]
FIT_PATH_hs1 = cfg_hs1["preprocessing"]["fitting"]["missing_noise_path"]
# probe 2
cfg_hs2, _ = get_config("silico_horvath", "concatenated/probe_2").values()
RAW_PATH_hs2 = cfg_hs2["dataeng"]["campaign"]["output"]["trace_file_path"]
PREP_PATH_hs2 = cfg_hs2["preprocessing"]["output"]["trace_file_path"]
FIT_PATH_hs2 = cfg_hs2["preprocessing"]["fitting"]["missing_noise_path"]
# probe 3
cfg_hs3, _ = get_config("silico_horvath", "concatenated/probe_3").values()
RAW_PATH_hs3 = cfg_hs3["dataeng"]["campaign"]["output"]["trace_file_path"]
PREP_PATH_hs3 = cfg_hs3["preprocessing"]["output"]["trace_file_path"]
FIT_PATH_hs3 = cfg_hs3["preprocessing"]["fitting"]["missing_noise_path"]

# Figure settings
FIG_SIZE = (1.5, 1.6)
COLOR_VIVO = np.array([153, 153, 153]) / 255
COLOR_SILI = np.array([228, 26, 28]) / 255
COLOR_BUCCI = np.array([55, 126, 184]) / 255
COLOR_NE = [1, 0.49, 0]
COLOR_MEDIAN = (1, 0.76, 0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2024-07-12 16:41:08,328 - root - utils.py - get_config - INFO - Reading experiment config.
2024-07-12 16:41:08,346 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-07-12 16:41:08,347 - root - utils.py - get_config - INFO - Reading experiment config.
2024-07-12 16:41:08,357 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-07-12 16:41:08,358 - root - utils.py - get_config - INFO - Reading experiment config.
2024-07-12 16:41:08,370 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-07-12 16:41:08,371 - root - utils.py - get_config - INFO - Reading experiment config.
2024-07-12 16:41:08,397 - root - utils.py - get_config - INFO - Reading experiment config. - done
2024-07-12 16:41:08,427 - root - utils.py - get_config - INFO - Reading experiment config.
2024-07-12 16:41:08,445 - root - utils.py - get_config - INFO - R

## Load dataset

In [3]:
def get_fit_prms(fit_path, layer):
    gain = np.load(fit_path + layer + ".npy", allow_pickle=True).item()["gain"]
    tuning = np.load(fit_path + layer + ".npy", allow_pickle=True).item()[
        "gain_adjustm"
    ]
    gain *= tuning
    missing_noise = np.load(fit_path + layer + ".npy", allow_pickle=True).item()[
        "missing_noise_rms"
    ]
    # report
    print(
        f"- {layer} gain: {np.round(gain)}, missing noise rms: {np.round(missing_noise,2)} uV"
    )

## Report

In [11]:
# memo: fits are matched to layers and depths

# Depth 1 (L1, 2/3)
print("Denser-Depth 1" + ":")
get_fit_prms(
    FIT_PATH_hs1,
    "L1",
)
get_fit_prms(FIT_PATH_hs1, "L2_3")

# Depth 2 (L4, 5)
print("\nDenser-Depth 2" + ":")
get_fit_prms(
    FIT_PATH_hs2,
    "L4",
)
get_fit_prms(FIT_PATH_hs2, "L5")

# Depth 2 (L6)
print("\nDenser-Depth 2" + ":")
get_fit_prms(
    FIT_PATH_hs3,
    "L6",
)

Denser-Depth 1:
- L1 gain: 12463.0, missing noise rms: 27.31 uV
- L2_3 gain: 12463.0, missing noise rms: 30.33 uV

Denser-Depth 2:
- L4 gain: 1949.0, missing noise rms: 37.39 uV
- L5 gain: 1949.0, missing noise rms: 37.94 uV

Denser-Depth 2:
- L6 gain: 20778.0, missing noise rms: 35.64 uV
